In [25]:
import pandas as pd

In [82]:
df=pd.read_csv('data/finalTrain.csv')


In [83]:
df=df.drop(labels=['ID'],axis=1)

In [84]:
import haversine as hs
from haversine import Unit

def dist_calculate(lat1,lon1,lat2,lon2):
    loc1=(lat1,lon1)
    loc2=(lat2,lon2)
    result=hs.haversine(loc1, loc2,unit=Unit.KILOMETERS)
    return result

for i in range(len(df)):
    df.loc[i,'distance']=dist_calculate(df.loc[i,'Restaurant_latitude'],
                                        df.loc[i,'Restaurant_longitude'],
                                        df.loc[i,'Delivery_location_latitude'],
                                        df.loc[i,'Delivery_location_longitude']                             
                                          )

In [86]:
df['time_diff']=(df['Time_Order_picked'].str.split(':').str[0].astype(float)*60+df['Time_Order_picked'].str.split(':').str[1].astype(float))-(df['Time_Orderd'].str.split(':').str[0].astype(float)*60+df['Time_Orderd'].str.split(':').str[1].astype(float))


In [88]:
df=df.drop(labels=['Time_Order_picked','Time_Orderd','Delivery_person_ID','Order_Date','Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude','Weather_conditions'],axis=1)

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45584 entries, 0 to 45583
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Delivery_person_Age      43730 non-null  float64
 1   Delivery_person_Ratings  43676 non-null  float64
 2   Road_traffic_density     44983 non-null  object 
 3   Vehicle_condition        45584 non-null  int64  
 4   Type_of_order            45584 non-null  object 
 5   Type_of_vehicle          45584 non-null  object 
 6   multiple_deliveries      44591 non-null  float64
 7   Festival                 45356 non-null  object 
 8   City                     44384 non-null  object 
 9   Time_taken (min)         45584 non-null  int64  
 10  distance                 45584 non-null  float64
 11  time_diff                35818 non-null  float64
dtypes: float64(5), int64(2), object(5)
memory usage: 4.2+ MB


In [90]:
X = df.drop(labels=['Time_taken (min)'],axis=1)
Y = df[['Time_taken (min)']]

In [91]:
X

,Delivery_person_Age,Delivery_person_Ratings,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,distance,time_diff
0,36.0,4.2,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,10.280596,15.0
1,21.0,4.7,High,1,Meal,motorcycle,1.0,No,Metropolitian,6.242327,10.0
2,23.0,4.7,Medium,1,Drinks,scooter,1.0,No,Metropolitian,13.787879,10.0
3,34.0,4.3,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,2.930262,10.0
4,24.0,4.7,Jam,1,Snack,scooter,1.0,No,Metropolitian,19.396645,15.0
...,...,...,...,...,...,...,...,...,...,...,...
45579,30.0,4.8,High,1,Meal,motorcycle,0.0,No,Metropolitian,1.489848,10.0
45580,21.0,4.6,Jam,0,Buffet,motorcycle,1.0,No,Metropolitian,11.007750,15.0
45581,30.0,4.9,Low,1,Drinks,scooter,0.0,No,Metropolitian,4.657202,15.0
45582,20.0,4.7,High,0,Snack,motorcycle,1.0,No,Metropolitian,6.232402,5.0


In [92]:
Y

,Time_taken (min)
0,46
1,23
2,21
3,20
4,41
...,...
45579,32
45580,36
45581,16
45582,26


In [93]:
categorical_cols = X.select_dtypes(include= 'object').columns
numerical_cols = X.select_dtypes(exclude= 'object').columns

In [94]:
num_columns

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Vehicle_condition',
       'multiple_deliveries', 'distance', 'time_diff'],
      dtype='object')

In [95]:
cat_columns

Index(['Road_traffic_density', 'Type_of_order', 'Type_of_vehicle', 'Festival',
       'City'],
      dtype='object')

In [96]:
X['Road_traffic_density'].unique()


array(['Jam', 'High', 'Medium', 'Low', nan], dtype=object)

In [97]:
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [98]:
Type_of_order_cat = ['Drinks', 'Buffet', 'Snack', 'Meal']
Type_of_vehicle_cat=['electric_scooter', 'scooter','bicycle', 'motorcycle']
Road_traffic_density_cat=['Low','Medium','High','Jam']
Weather_conditions_cat=['Sunny','Stormy','Sandstorms','Windy','Fog','Cloudy']
City_cat=['Urban','Metropolitian','Semi-Urban']
Festival_c=['No','Yes']

In [102]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
    ]
)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[Type_of_order_cat,Type_of_vehicle_cat,Weather_conditions_cat,Road_traffic_density_cat,City_cat,Festival_c])),
    ('scaler',StandardScaler())
    ]
)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,num_columns),
('cat_pipeline',cat_pipeline,cat_columns)
])

In [103]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Vehicle_condition',
       'multiple_deliveries', 'distance', 'time_diff'],
      dtype='object')),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_f...
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Drinks',
                                                                              'Buffet',
                                                                              'Snack',
                                                                              'Meal'],
                                                                             ['electric_scooter',
                                                                              'scooter',
                                                                              'bicycle',
                                                                              'motorcycle'],
                                                                             ['Low',
                                                                              'Medium',
                                                                              'High',
                                                                              'Jam'],
                                                                             ['Urban',
                                                                              'Metropolitian',
                                                                              'Semi-Urban'],
                                                                             ['No',
                                                                              'Yes']])),
                                                 ('scaler', StandardScaler())]),
                                 Index(['Road_traffic_density', 'Type_of_order', 'Type_of_vehicle', 'Festival',
       'City'],
      dtype='object'))])

In [105]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=20)

In [107]:
preprocessor.fit_transform(X_train)

ValueError: Found unknown categories ['High', 'Jam', 'Medium', 'Low'] in column 0 during fit

In [104]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())

X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

ValueError: Found unknown categories ['High', 'Jam', 'Medium', 'Low'] in column 0 during fit

In [229]:
X_train.head()

,Delivery_person_Age,Delivery_person_Ratings,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,distance,time_diff
2389,34.0,4.7,Windy,Low,2,Snack,scooter,1.0,No,Metropolitian,3.087893,10.0
37768,31.0,4.2,Windy,Jam,1,Snack,motorcycle,0.0,No,Metropolitian,12.099040,NaN
21005,38.0,4.6,Sunny,Low,2,Snack,electric_scooter,1.0,No,Metropolitian,17.138130,5.0
14414,32.0,4.8,Sandstorms,Low,2,Snack,scooter,0.0,No,Metropolitian,10.881093,10.0
17583,33.0,4.7,Fog,Medium,2,Meal,electric_scooter,1.0,No,Urban,12.420362,NaN


In [230]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [231]:

regression=LinearRegression()
regression.fit(X_train,y_train)

ValueError: could not convert string to float: 'Windy'

In [ ]:
# Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

In [ ]:

import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')